In [1]:
import pandas as pd

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

In [2]:
input_ = pd.read_excel('./9.url.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['No'] = input_['No'].astype(int)

print(f'总数量：{len(input_)}')
input_['Url'].tolist()

总数量：97463


['https://www.dormanproducts.com/p-74380-949-006.aspx',
 'https://www.dormanproducts.com/p-129165-918-042.aspx',
 'https://www.dormanproducts.com/p-86802-917-954.aspx',
 'https://www.dormanproducts.com/p-129181-918-524.aspx',
 'https://www.dormanproducts.com/p-80124-420-181.aspx',
 'https://www.dormanproducts.com/p-139014-15011.aspx',
 'https://www.dormanproducts.com/p-12229-700-081.aspx',
 'https://www.dormanproducts.com/p-73288-635-118.aspx',
 'https://www.dormanproducts.com/p-84682-46104.aspx',
 'https://www.dormanproducts.com/p-5974-82572.aspx',
 'https://www.dormanproducts.com/p-20821-90460.aspx',
 'https://www.dormanproducts.com/p-98532-916-885.aspx',
 'https://www.dormanproducts.com/p-105838-917-143cd.aspx',
 'https://www.dormanproducts.com/p-19763-3626191.aspx',
 'https://www.dormanproducts.com/p-54790-917-016-t.aspx',
 'https://www.dormanproducts.com/p-98572-917-992cd.aspx',
 'https://www.dormanproducts.com/p-79440-917-414.aspx',
 'https://www.dormanproducts.com/p-85709-42315.

In [3]:
a = 0

a

0

In [4]:
crawler_status = 'error'

crawler_status

'error'

In [5]:
import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

b = 0
while True:
    b += 1
    print(b)
    try:
        resp = requests.get(input_.loc[a, 'Url'],
                            headers=get_header(),
                            proxies=get_proxy(),
                            timeout=(10, 10))

        if resp.status_code == 200:
            break
    except:
        continue

resp

1


<Response [200]>

In [6]:
from bs4 import BeautifulSoup
from lxml import etree

soup = BeautifulSoup(resp.text, 'lxml')
html = etree.HTML(str(soup))

with open('./part.html', 'w', encoding='UTF-8') as file:
    file.write(str(soup))

print(soup.prettify())

<!DOCTYPE html>
<html lang="en">
 <head>
  <title>
   Air Suspension Line Repair Kit | 949-006 | Air Suspension Line Repair Kit | Dorman Products
  </title>
  <meta charset="utf-8"/>
  <meta content="no-cache, no-store, must-revalidate" http-equiv="cache-control"/>
  <meta content="no-cache" http-equiv="pragma"/>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="-1" http-equiv="expires"/>
  <meta content="Dorman Products - 949-006 : Air Suspension Line Repair Kit. This air suspension compressor line repair kit is designed to match the original components required to repair the air suspension compressor on specified vehicles. It's made of quality materials for reliable service." name="description"/>
  <meta content="949-006,  Suspension, Air, Compressor, Line, Repair, Kit, Kits, Lines, Compressors, Air Suspension Line Kit, Air Ride Compressor Line Repair Kit, Air Suspension Line Repair Union, Air Ride Compressor Line Repair Union., Air Suspension Co

In [7]:
type_ = html.xpath('//h1/span[@id="lblPartTypeName"]/text()')[0]

type_

' Air Suspension Line Repair Kit'

In [8]:
i, dict_src = 0, {}
list_src = list(reversed([href.strip() for href in html.xpath('//a[@data-zoom-id="ProductPic"]/@href')]))
for src in list_src:
    if src.startswith('http'):
        dict_src[str(i)] = src
        i += 1

dict_src

{'0': 'https://static.dormanproducts.com/images/product/medium/949-006-007.JPG'}

In [9]:
part_number = html.xpath('//span[@id="lblProductName"]/text()')[0].strip()

part_number

'949-006'

In [10]:
description = html.xpath('//span[@id="lblProductDesc"]/text()')[0].strip()

description

'Air Suspension Line Repair Kit'

In [11]:
application_summary = '\n'.join(html.xpath('//div[@id="divAppSummary"]/p/text()')).strip()

application_summary

'Cadillac 2006-02, Chevrolet 2006-01, GMC 2006-01'

In [12]:
brand = html.xpath('//span[@id="lblBrandName"]/text()')[0].strip()

brand

'Dorman - OE Solutions'

In [13]:
vehicle_url = html.xpath('//input[@id="hidfDetailApp"]/@value')
vehicle_url = f'''https://www.dormanproducts.com/{vehicle_url[0].strip()}''' if vehicle_url else ''

vehicle_url

'https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=74380&PartType=%20Air%20Suspension%20Line%20Repair%20Kit'

In [14]:
dict_specification = {}
list_tr = html.xpath('//section[@id="productSpec"]/div/table/tr')
for i, tr in zip(range(len(list_tr)), list_tr):
    dict_specification[str(i)] = {tr.xpath('./th/text()')[0].strip().removesuffix(':').strip(): tr.xpath('./td/text()')[0].strip()}

dict_specification

{'0': {'Color': 'Black; Clear'},
 '1': {'Connector Quantity': 'M6 -1.0'},
 '2': {'Connector Type': '4 90 deg Connector; 2 Straight Connector Heads'},
 '3': {'Length': '86 in; 18 in'},
 '4': {'Material': 'Nylon'},
 '5': {'Package Contents': '1 Air Suspension Compressor Line Repair Kit'},
 '6': {'Retainer Clips Included': 'Yes'},
 '7': {'T Joint Included': 'Yes'},
 '8': {'Tube Outside Diameter': '0.13 in'}}

In [15]:
list_oem = [text.strip() for text in html.xpath('//section[@id="productOE"]/div/table/tbody/tr/th/text()')]
oem = ';'.join(sorted(list(set(list_oem)), key=list_oem.index))

oem

'88987900'

In [16]:
import json

df_temp = pd.DataFrame([{'No': input_.loc[a, 'No'],
                         'Type': type_,
                         'Part Number': part_number,
                         'Brand': brand,
                         'Description': description,
                         'Application Summary': application_summary,
                         'OEM': oem,
                         'Url': input_.loc[a, 'Url'],
                         'Vehicle Url': vehicle_url,
                         'Json_Src': json.dumps(dict_src),
                         'Json_Specification': json.dumps(dict_specification)}])

output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

output_correct

,No,Type,Part Number,Brand,Description,Application Summary,OEM,Url,Vehicle Url,Json_Src,Json_Specification
0,1,Air Suspension Line Repair Kit,949-006,Dorman - OE Solutions,Air Suspension Line Repair Kit,"Cadillac 2006-02, Chevrolet 2006-01, GMC 2006-01",88987900,https://www.dormanproducts.com/p-74380-949-006...,https://www.dormanproducts.com/itemdetailapp.a...,"{""0"": ""https://static.dormanproducts.com/image...","{""0"": {""Color"": ""Black; Clear""}, ""1"": {""Connec..."


In [17]:
crawler_status = 'ok'

crawler_status

'ok'

In [18]:
if crawler_status == 'error':
    df_temp = pd.DataFrame([{'No': input_.loc[a, 'No'],
                             'Url': input_.loc[a, 'Url']}])

    output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

output_error

""


In [19]:
if not output_correct.empty:
    output_correct = output_correct.sort_values(by=['No'],
                                                ascending=[True],
                                                ignore_index=True)
    output_correct.to_excel('./test_part_1.xlsx', index=False)

output_correct

,No,Type,Part Number,Brand,Description,Application Summary,OEM,Url,Vehicle Url,Json_Src,Json_Specification
0,1,Air Suspension Line Repair Kit,949-006,Dorman - OE Solutions,Air Suspension Line Repair Kit,"Cadillac 2006-02, Chevrolet 2006-01, GMC 2006-01",88987900,https://www.dormanproducts.com/p-74380-949-006...,https://www.dormanproducts.com/itemdetailapp.a...,"{""0"": ""https://static.dormanproducts.com/image...","{""0"": {""Color"": ""Black; Clear""}, ""1"": {""Connec..."


In [20]:
if not output_error.empty:
    output_error = output_error.sort_values(by=['No'],
                                            ascending=[True],
                                            ignore_index=True)
    output_error.to_excel('./test_part_1_error.xlsx', index=False)

output_error

""
